# Trying out Bayesian Flow Networks

Trying out the discrete case.

In [1]:
from jaxtyping import install_import_hook

#with install_import_hook("train_and_sample", "beartype.beartype"):
import train_and_sample as tas
from example_data import corrupt_string

In [2]:
import jax.numpy as jnp
import jax.random as jr
import flax.linen as nn
import jax
import optax

In [3]:
d = 10
k = 5
dod = tas.DiscreteOutputDistribution(k, d)

thetas_example = jr.uniform(jr.PRNGKey(0), (k, d))
variables = dod.init(jr.PRNGKey(0), thetas_example, 0.5)
out = dod.apply(variables, thetas_example, 0.5)

In [6]:
variables["params"]

{'InnerNetwork_0': {'category_mixer': {'kernel': Array([[-0.09161296, -0.6874112 , -0.30309632, -0.44880354,  0.35827217],
          [-0.17161633,  0.64222443, -0.539467  ,  0.9133116 , -0.261737  ],
          [ 0.41200006, -0.00829238, -0.1983576 ,  0.867119  , -0.12587032],
          [-0.22224309, -0.5260398 , -0.06105281,  0.31751642,  0.39841345],
          [-0.0162582 ,  0.01465106, -0.37392122,  0.4502149 ,  0.51043797]],      dtype=float32),
   'bias': Array([0., 0., 0., 0., 0.], dtype=float32)},
  'position_mixer': {'kernel': Array([[-3.84128511e-01,  5.78072481e-02, -1.25126064e-01,
            6.29644096e-02,  8.52497295e-02,  5.61563205e-03,
           -6.76987618e-02, -2.67720539e-02, -6.01685226e-01,
            9.22407657e-02],
          [ 1.90649167e-01,  2.42735565e-01, -1.21889926e-01,
           -5.27845144e-01, -1.74039081e-01,  5.62949739e-02,
           -4.03896153e-01, -1.52295455e-01,  6.38252199e-01,
            3.85828912e-01],
          [-8.90800953e-02,  2.91

In [4]:
key = jr.PRNGKey(0)
key, subkey = jr.split(key)
x = jr.randint(key, shape=(d,), minval=0, maxval=k)
tas.loss(x, variables, dod, 0.5, key=subkey)

Array(5.449535, dtype=float32)

In [5]:
key, subkey = jr.split(key)
tas.sample(variables, dod, 0.5, 100, key=subkey)

Array([0, 4, 0, 2, 3, 3, 0, 3, 4, 4], dtype=int32)

### Trying Training

In [7]:
example_data = corrupt_string("elisroberts", 100, 0.15)
num_cats = 26
d = len(example_data[0])

In [9]:
# Initialize model
model = tas.DiscreteOutputDistribution(num_cats, d)
thetas_prior = jnp.ones((num_cats, d)) / num_cats

variables = model.init(jr.PRNGKey(0), thetas_prior, 1.0)

In [10]:
optim = optax.adam(1e-3)
opt_state = optim.init(variables)

In [ ]:
def compute_loss(model, )


def make_step(model, g, label, optim, opt_state):
    """Calculate loss & grad and update model according to optimiser."""
    loss, grads = compute_loss(model, g, label)
    updates, opt_state = optim.update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state